In [1]:
# Coral Bleaching Images

In [2]:
import kagglehub
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report


/Users/carloschutz/.pyenv/versions/3.10.6/envs/ReefSight-Project/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:

# Download Kaggle bleaching data
path = kagglehub.dataset_download("pnminh95/global-bleaching-environmental")
df = pd.read_csv(f"{path}/global_bleaching_environmental.csv")

print(df.head())
print(df.columns)

100%|██████████| 2.17M/2.17M [00:00<00:00, 3.48MB/s]

Extracting files...


   Site_ID  Sample_ID Data_Source  Latitude_Degrees  Longitude_Degrees  \
0     2501   10324336      Donner            23.163           -82.5260   
1     3467   10324754      Donner           -17.575          -149.7833   
2     1794   10323866      Donner            18.369           -64.5640   
3     8647   10328028      Donner            17.760           -64.5680   
4     8648   10328029      Donner            17.769           -64.5830   

  Ocean_Name Reef_ID            Realm_Name  \
0   Atlantic      nd     Tropical Atlantic   
1    Pacific      nd  Eastern Indo-Pacific   
2   Atlantic      nd     Tropical Atlantic   
3   Atlantic      nd     Tropical Atlantic   
4   Atlantic      nd     Tropical Atlantic   

                               Ecoregion_Name      Country_Name  ...  \
0                     Cuba and Cayman Islands              Cuba  ...   
1            Society Islands French Polynesia  French Polynesia  ...   
2  Hispaniola Puerto Rico and Lesser Antilles    United Kingdo

/var/folders/pd/49tggkj16kj_jxk0fmm8f0b00000gn/T/ipykernel_6290/2270386899.py:3: DtypeWarning: Columns (13,15,24) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f"{path}/global_bleaching_environmental.csv")


In [8]:
import kagglehub
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# Load & explore
path = kagglehub.dataset_download("pnminh95/global-bleaching-environmental")
df = pd.read_csv(f"{path}/global_bleaching_environmental.csv")

# AUTO-FIND numeric columns (safe!)
numeric_cols = df.select_dtypes(include=['number']).columns.tolist()
print(f"Found {len(numeric_cols)} numeric columns: {numeric_cols}")

# Find target (columns with 'bleach', 'bleaching', 'health' in name)
target_candidates = [col for col in df.columns if any(word in col.lower() for word in ['bleach', 'bleaching', 'health'])]
print(f"Target candidates: {target_candidates}")

# Use first numeric cols + first target candidate
features = numeric_cols[:6]  # First 6 numeric columns
target = target_candidates[0] if target_candidates else 'bleaching_severity'

print(f"\n✅ Using features: {features}")
print(f"✅ Target: {target}")

# Train
X = df[features].fillna(0)

# Safely convert the target column to numeric before binning.
# Many columns are object dtype with values like 'nd' — coerce to numeric, map common categorical labels if needed.
y_col = pd.to_numeric(df[target].replace('nd', pd.NA), errors='coerce')

# If numeric conversion yields all NaN, try mapping common categorical labels to numeric classes
if y_col.isna().all():
	mapping = {
		'none': 0, 'no': 0, 'low': 0, 'mild': 0, 'partial': 0, 'partial bleaching': 0,
		'some': 0,
		'yes': 1, 'high': 1, 'severe': 1, 'bleached': 1, 'major': 1
	}
	y_col = df[target].astype(str).str.lower().str.strip().map(mapping)

# Fallback: fill remaining NaNs with 0 (assume low/no bleaching). Then bin into two classes.
y = pd.cut(y_col.fillna(0), bins=2, labels=[0,1]).astype(int)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
model = RandomForestClassifier(n_estimators=100)
model.fit(X_train, y_train)

print("✅ Model trained!")


/var/folders/pd/49tggkj16kj_jxk0fmm8f0b00000gn/T/ipykernel_6290/970264410.py:8: DtypeWarning: Columns (13,15,24) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f"{path}/global_bleaching_environmental.csv")


Found 8 numeric columns: ['Site_ID', 'Sample_ID', 'Latitude_Degrees', 'Longitude_Degrees', 'Cyclone_Frequency', 'Date_Day', 'Date_Month', 'Date_Year']
Target candidates: ['Bleaching_Level', 'Percent_Bleaching', 'Bleaching_Comments']

✅ Using features: ['Site_ID', 'Sample_ID', 'Latitude_Degrees', 'Longitude_Degrees', 'Cyclone_Frequency', 'Date_Day']
✅ Target: Bleaching_Level
✅ Model trained!


In [9]:
import kagglehub
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import numpy as np

print("🚀 Downloading coral bleaching dataset...")
path = kagglehub.dataset_download("pnminh95/global-bleaching-environmental")
df = pd.read_csv(f"{path}/global_bleaching_environmental.csv")

print("\n📋 Dataset loaded!")
print(f"Shape: {df.shape}")
print("\n📊 Columns:")
print(df.columns.tolist())

# Find best features automatically
numeric_cols = df.select_dtypes(include=['number']).columns.tolist()
bleach_cols = [col for col in numeric_cols if any(word in col.lower() for word in ['bleach', 'bleaching', 'severity', 'health'])]

print(f"\n🔍 Numeric features: {len(numeric_cols)}")
print(f"🎯 Bleaching columns: {bleach_cols}")

# Use safe feature selection
if len(numeric_cols) >= 5:
    features = numeric_cols[:6]  # Top 6 numeric
else:
    features = numeric_cols

if bleach_cols:
    target = bleach_cols[0]
else:
    # Fallback: create binary target from first numeric col
    target = numeric_cols[0]
    df['bleached'] = (df[target] > df[target].median()).astype(int)
    target = 'bleached'

print(f"\n✅ Features: {features}")
print(f"✅ Target: {target}")

# Prepare data
X = df[features].fillna(0)
y = pd.cut(df[target].fillna(0), bins=2, labels=[0,1]).astype(int)

print(f"\n📈 Training model...")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Results
y_pred = model.predict(X_test)
print("\n📊 Model Performance:")
print(classification_report(y_test, y_pred))

# Test prediction
test_site = np.array([[30.5, 1.2, 0.1, -8.123, 115.234, 5.0]])  # SST, turbidity, etc.
risk = model.predict_proba(test_site)[0][1]
print(f"\n🪸 New site bleaching risk: {risk:.1%}")


🚀 Downloading coral bleaching dataset...


/var/folders/pd/49tggkj16kj_jxk0fmm8f0b00000gn/T/ipykernel_6290/1422218733.py:10: DtypeWarning: Columns (13,15,24) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f"{path}/global_bleaching_environmental.csv")



📋 Dataset loaded!
Shape: (41361, 62)

📊 Columns:
['Site_ID', 'Sample_ID', 'Data_Source', 'Latitude_Degrees', 'Longitude_Degrees', 'Ocean_Name', 'Reef_ID', 'Realm_Name', 'Ecoregion_Name', 'Country_Name', 'State_Island_Province_Name', 'City_Town_Name', 'Site_Name', 'Distance_to_Shore', 'Exposure', 'Turbidity', 'Cyclone_Frequency', 'Date_Day', 'Date_Month', 'Date_Year', 'Depth_m', 'Substrate_Name', 'Percent_Cover', 'Bleaching_Level', 'Percent_Bleaching', 'ClimSST', 'Temperature_Kelvin', 'Temperature_Mean', 'Temperature_Minimum', 'Temperature_Maximum', 'Temperature_Kelvin_Standard_Deviation', 'Windspeed', 'SSTA', 'SSTA_Standard_Deviation', 'SSTA_Mean', 'SSTA_Minimum', 'SSTA_Maximum', 'SSTA_Frequency', 'SSTA_Frequency_Standard_Deviation', 'SSTA_FrequencyMax', 'SSTA_FrequencyMean', 'SSTA_DHW', 'SSTA_DHW_Standard_Deviation', 'SSTA_DHWMax', 'SSTA_DHWMean', 'TSA', 'TSA_Standard_Deviation', 'TSA_Minimum', 'TSA_Maximum', 'TSA_Mean', 'TSA_Frequency', 'TSA_Frequency_Standard_Deviation', 'TSA_Frequ

/Users/carloschutz/.pyenv/versions/3.10.6/envs/ReefSight-Project/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
